＃ 问题回答

本笔记涵盖了如何评估通用问题回答问题。这是一个情况，您有一个包含问题及其对应的真实答案的示例，您想衡量语言模型在回答这些问题方面的表现如何。

## 设置

出于演示目的，我们将仅评估一个仅评估模型内部知识的简单问题回答系统。请参阅其他笔记本，以了解它如何在模型未经过训练的数据上进行问题回答的示例。

In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [2]:
prompt = PromptTemplate(template="Question: {question}\nAnswer:", input_variables=["question"])

In [3]:
llm = OpenAI(model_name="text-davinci-003", temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)

## 示例
为此，我们将仅使用两个简单的硬编码示例，但请参阅其他笔记本，了解如何获得和/或生成这些示例的提示。

In [4]:
examples = [
    {
        "question": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?",
        "answer": "11"
    },
    {
        "question": 'Is the following sentence plausible? "Joao Moutinho caught the screen pass in the NFC championship."',
        "answer": "No"
    }
]

## 预测

现在我们可以制作和检查这些问题的预测。

In [5]:
predictions = chain.apply(examples)

In [6]:
predictions

[{'text': ' 11 tennis balls'},
 {'text': ' No, this sentence is not plausible. Joao Moutinho is a professional soccer player, not an American football player, so it is not likely that he would be catching a screen pass in the NFC championship.'}]

## 评估

我们可以看到，如果我们仅对答案进行精确匹配（`11`和`No`），它们将无法匹配语言模型的回答。但是，从语义上讲，语言模型在两种情况下都是正确的。为了解决这个问题，我们可以使用语言模型本身来评估答案。

In [7]:
from langchain.evaluation.qa import QAEvalChain

In [8]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(examples, predictions, question_key="question", prediction_key="text")

In [9]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + eg['question'])
    print("Real Answer: " + eg['answer'])
    print("Predicted Answer: " + predictions[i]['text'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
Real Answer: 11
Predicted Answer:  11 tennis balls
Predicted Grade:  CORRECT

Example 1:
Question: Is the following sentence plausible? "Joao Moutinho caught the screen pass in the NFC championship."
Real Answer: No
Predicted Answer:  No, this sentence is not plausible. Joao Moutinho is a professional soccer player, not an American football player, so it is not likely that he would be catching a screen pass in the NFC championship.
Predicted Grade:  CORRECT



## 自定义提示

您也可以自定义使用的提示。这是一个用分数从0到10提示它的示例。
自定义提示需要3个输入变量：“查询”、“答案”和“结果”。其中，“查询”是问题，“答案”是真实答案，“结果”是预测答案。

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_PROMPT_TEMPLATE = """You are an expert professor specialized in grading students' answers to questions.
You are grading the following question:
{query}
Here is the real answer:
{answer}
You are grading the following predicted answer:
{result}
What grade do you give from 0 to 10, where 0 is the lowest (very low similarity) and 10 is the highest (very high similarity)?
"""

PROMPT = PromptTemplate(input_variables=["query", "answer", "result"], template=_PROMPT_TEMPLATE)

In [ ]:
evalchain = QAEvalChain.from_llm(llm=llm,prompt=PROMPT)
evalchain.evaluate(examples, predictions, question_key="question", answer_key="answer", prediction_key="text")

## 无基础真相的评估
可以在没有基础真相的情况下评估问题回答系统。您需要一个反映LLM用于回答问题的信息的"context"输入。这个上下文可以通过任何检索系统获得。以下是它的工作原理的例子：

In [ ]:
context_examples = [
    {
        "question": "How old am I?",
        "context": "I am 30 years old. I live in New York and take the train to work everyday.",
    },
    {
        "question": 'Who won the NFC championship game in 2023?"',
        "context": "NFC Championship Game 2023: Philadelphia Eagles 31, San Francisco 49ers 7"
    }
]
QA_PROMPT = "Answer the question based on the  context\nContext:{context}\nQuestion:{question}\nAnswer:"
template = PromptTemplate(input_variables=["context", "question"], template=QA_PROMPT)
qa_chain = LLMChain(llm=llm, prompt=template)
predictions = qa_chain.apply(context_examples)

In [12]:
predictions

[{'text': 'You are 30 years old.'},
 {'text': ' The Philadelphia Eagles won the NFC championship game in 2023.'}]

In [9]:
from langchain.evaluation.qa import ContextQAEvalChain
eval_chain = ContextQAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(context_examples, predictions, question_key="question", prediction_key="text")

In [13]:
graded_outputs

[{'text': ' CORRECT'}, {'text': ' CORRECT'}]

## 与其他评估指标比较
我们可以将获得的评估结果与其他常见评估指标进行比较。为此，让我们从HuggingFace的`evaluate`包中加载一些评估指标。

In [10]:
# Some data munging to get the examples in the right format
for i, eg in enumerate(examples):
    eg['id'] = str(i)
    eg['answers'] = {"text": [eg['answer']], "answer_start": [0]}
    predictions[i]['id'] = str(i)
    predictions[i]['prediction_text'] = predictions[i]['text']

for p in predictions:
    del p['text']

new_examples = examples.copy()
for eg in new_examples:
    del eg ['question']
    del eg['answer']

In [11]:
from evaluate import load
squad_metric = load("squad")
results = squad_metric.compute(
    references=new_examples,
    predictions=predictions,
)

In [12]:
results

{'exact_match': 0.0, 'f1': 28.125}